In [ ]:
import pandas as pd
import folium

In [ ]:
DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv')
MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv')
MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv')
PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv')
TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv')
TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv')
TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv')
TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv')

In [ ]:
# 컬럼 이름 공백 제거
def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]
datasets = [clean_columns(df) for df in datasets]

In [ ]:
# 모든 데이터 합치기
all_data = pd.concat(datasets, ignore_index=True)

In [ ]:
# Elem 코드와 파라미터명 매핑
param_map = {
    2: 'DP01',
    7: 'MNVP',
    6: 'MSLP',
    1: 'PRCP',
    5: 'TAVG',
    3: 'TMAX',
    4: 'TMIN',
    8: 'TSUN'
}

In [ ]:
all_data['Parameter'] = all_data['Elem'].map(param_map)

In [ ]:
all_data

In [ ]:
# 월별 컬럼 리스트 (Jan, Feb, ... Dec)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 특정 달, 특정 Parameter 조건 필터 예: TMAX가 20~25 사이인 지역 찾기 (7월 예시)
selected_param = 'TMAX'
selected_month = 'Jul'
min_val, max_val = 20, 25

filtered = all_data[(all_data['Parameter'] == selected_param) & 
                    (all_data[selected_month] >= min_val) & 
                    (all_data[selected_month] <= max_val)]

In [ ]:
import plotly.express as px

# 지도 시각화 예시 - Plotly scatter_geo 사용
fig = px.scatter_geo(filtered,
                     lat='Latitude',
                     lon='Longitude',
                     hover_name='Station',
                     hover_data={'Country': True, selected_month: True},
                     color=selected_month,
                     projection='natural earth')

# 조건에 맞는 지점 표시(붉은 깃발 아이콘)
# Plotly에서는 마커 심볼 변경 가능. 레드 컬러와 특정 심볼(예: "flag")을 지정.
fig.update_traces(marker=dict(size=8, color='red', symbol='triangle-up'))
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, widgets

# 데이터 로드 및 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

param_map = {
    2: 'DP01',
    7: 'MNVP',
    6: 'MSLP',
    1: 'PRCP',
    5: 'TAVG',
    3: 'TMAX',
    4: 'TMIN',
    8: 'TSUN'
}
all_data['Parameter'] = all_data['Elem'].map(param_map)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 슬라이더 기반 업데이트 함수
def update_plot(selected_month, selected_params, min_val, max_val):
    # 여러 파라미터 필터링
    filtered = all_data[
        (all_data['Parameter'].isin(selected_params)) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    # Plotly 지도 생성
    fig = px.scatter_geo(
        filtered,
        lat='Latitude',
        lon='Longitude',
        hover_name='Station',
        hover_data={selected_month: True, 'Parameter': True},
        projection='natural earth'
    )
    
    # 마커 및 스타일 업데이트
    fig.update_traces(
        mode='text',
        text='🚩',
        textfont=dict(size=12)
    )
    fig.update_geos(
        showland=True, landcolor="rgb(229, 229, 229)",
        showocean=True, oceancolor="rgb(204, 229, 255)",
        showcountries=True, showframe=False
    )
    fig.update_layout(
        title=f"Filtered Data for {selected_month} ({min_val} - {max_val})",
        margin={"r": 0, "t": 30, "l": 0, "b": 0}
    )
    
    fig.show()

# 위젯 설정
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jul',
    description='Month:'
)

param_selection = widgets.SelectMultiple(
    options=list(param_map.values()),
    value=['TMAX'],
    description='Parameters:'
)

min_slider = widgets.IntSlider(value=10, min=0, max=50, step=1, description='Min Val:')
max_slider = widgets.IntSlider(value=30, min=0, max=50, step=1, description='Max Val:')

# 인터랙티브 위젯
interact(
    update_plot,
    selected_month=month_dropdown,
    selected_params=param_selection,
    min_val=min_slider,
    max_val=max_slider
)

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, widgets

# 데이터 로드 및 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

param_map = {
    2: 'DP01',
    7: 'MNVP',
    6: 'MSLP',
    1: 'PRCP',
    5: 'TAVG',
    3: 'TMAX',
    4: 'TMIN',
    8: 'TSUN'
}
all_data['Parameter'] = all_data['Elem'].map(param_map)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 슬라이더 범위를 업데이트하는 함수
def update_sliders(selected_month, selected_param):
    # 선택한 월과 파라미터로 데이터 필터링
    filtered = all_data[all_data['Parameter'] == selected_param]
    min_value = filtered[selected_month].min()
    max_value = filtered[selected_month].max()
    return min_value, max_value

# 데이터 필터링 및 지도 업데이트 함수
def update_plot(selected_month, selected_param, min_val, max_val):
    # 선택한 월과 파라미터로 데이터 필터링
    filtered = all_data[
        (all_data['Parameter'] == selected_param) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    # Plotly 지도 생성
    fig = px.scatter_geo(
        filtered,
        lat='Latitude',
        lon='Longitude',
        hover_name='Station',
        hover_data={selected_month: True, 'Parameter': True},
        projection='natural earth'
    )

    # 마커 및 스타일 업데이트
    fig.update_traces(
        mode='text',
        text='🚩',
        textfont=dict(size=12)
    )
    fig.update_geos(
        showland=True, landcolor="rgb(229, 229, 229)",
        showocean=True, oceancolor="rgb(204, 229, 255)",
        showcountries=True, showframe=False
    )
    fig.update_layout(
        title=f"{selected_month} - {selected_param} between {min_val} and {max_val}",
        margin={"r": 0, "t": 30, "l": 0, "b": 0}
    )

    fig.show()

# 위젯 구성
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jul',
    description='Month:'
)

param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='TMAX',
    description='Parameter:'
)

min_slider = widgets.IntSlider(value=10, min=0, max=50, step=1, description='Min Val:')
max_slider = widgets.IntSlider(value=30, min=0, max=50, step=1, description='Max Val:')

# 위젯 간 동적 연동
def on_param_or_month_change(change):
    # 선택한 Month와 Parameter 가져오기
    selected_month = month_dropdown.value
    selected_param = param_dropdown.value
    
    # 슬라이더의 min, max 업데이트
    min_value, max_value = update_sliders(selected_month, selected_param)
    min_slider.min = int(min_value)
    min_slider.max = int(max_value)
    max_slider.min = int(min_value)
    max_slider.max = int(max_value)
    min_slider.value = int(min_value)
    max_slider.value = int(max_value)

# Month와 Parameter 변경 이벤트 리스너
month_dropdown.observe(on_param_or_month_change, names='value')
param_dropdown.observe(on_param_or_month_change, names='value')

# 최종 인터랙티브 구성
interact(
    update_plot,
    selected_month=month_dropdown,
    selected_param=param_dropdown,
    min_val=min_slider,
    max_val=max_slider
)

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, widgets

# 데이터 로드 및 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# 파라미터 매핑 및 이름 수정
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

unit_map = {
    'Precipitation': 'mm',
    'Number of Days with Precipitation ≥ 1 mm': '#Days',
    'Mean Daily Maximum Temperature': 'degC',
    'Mean Daily Minimum Temperature': 'degC',
    'Mean Daily Mean Temperature': 'degC',
    'Mean Sea Level Pressure': 'hPa',
    'Mean Vapor Pressure': 'hPa',
    'Total Number of Hours of Sunshine': 'Hours'
}

# 데이터에 Parameter 이름 적용
all_data['Parameter'] = all_data['Elem'].map(param_map)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 슬라이더 범위를 업데이트하는 함수
def update_sliders(selected_month, selected_param):
    # # 선택한 월과 파라미터로 데이터 필터링
    # filtered = all_data[all_data['Parameter'] == selected_param]
    # min_value = filtered[selected_month].min()
    # max_value = filtered[selected_month].max()
    # # 단위를 추출하여 슬라이드 값에 반영
    # param_name = selected_param.split(' (')[0]  # 이름에서 단위 제거
    # unit = unit_map[param_name]
    # return min_value, max_value, unit
    # NaN을 무시한 최소/최대 값 계산
    filtered = all_data[all_data['Parameter'] == selected_param]
    min_value = filtered[selected_month].dropna().min()  # NaN 무시
    max_value = filtered[selected_month].dropna().max()
    param_name = selected_param.split(' (')[0]  # 이름에서 단위 제거
    unit = unit_map[param_name]
    return min_value, max_value, unit


# 데이터 필터링 및 지도 업데이트 함수
def update_plot(selected_month, selected_param, min_val, max_val):
    # 선택한 월과 파라미터로 데이터 필터링
    filtered = all_data[
        (all_data['Parameter'] == selected_param) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    # Plotly 지도 생성
    fig = px.scatter_geo(
        filtered,
        lat='Latitude',
        lon='Longitude',
        hover_name='Station',
        hover_data={selected_month: True, 'Parameter': True}
    )

    # 마커 및 스타일 업데이트
    fig.update_traces(
        mode='text',
        text='🚩',
        textfont=dict(size=12)
    )
    fig.update_geos(
        showland=True, landcolor="rgb(229, 229, 229)",
        showocean=True, oceancolor="rgb(204, 229, 255)",
        showcountries=True, showframe=False
    )
    fig.update_layout(
        title=f"{selected_month} - {selected_param} between {min_val} and {max_val}",
        margin={"r": 0, "t": 30, "l": 0, "b": 0}
    )

    fig.show()

# 위젯 구성
month_dropdown = widgets.Dropdown(
    options=months,
    value='Dec',
    description='Month:'
)

param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Mean Daily Maximum Temperature (degC)',
    description='Parameter:'
)

min_slider = widgets.IntSlider(value=15, min=0, max=50, step=1, description='Min Val:')
max_slider = widgets.IntSlider(value=20, min=0, max=50, step=1, description='Max Val:')

# 위젯 간 동적 연동
def on_param_or_month_change(change):
    # 선택한 Month와 Parameter 가져오기
    selected_month = month_dropdown.value
    selected_param = param_dropdown.value
    
    # 슬라이더의 min, max, 단위 업데이트
    min_value, max_value, unit = update_sliders(selected_month, selected_param)
    min_slider.min = int(min_value)
    min_slider.max = int(max_value)
    max_slider.min = int(min_value)
    max_slider.max = int(max_value)
    min_slider.value = int(min_value)
    max_slider.value = int(max_value)
    min_slider.description = f"Min Val ({unit})"
    max_slider.description = f"Max Val ({unit})"

# Month와 Parameter 변경 이벤트 리스너
month_dropdown.observe(on_param_or_month_change, names='value')
param_dropdown.observe(on_param_or_month_change, names='value')

# 최종 인터랙티브 구성
interact(
    update_plot,
    selected_month=month_dropdown,
    selected_param=param_dropdown,
    min_val=min_slider,
    max_val=max_slider
)

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, widgets
import numpy as np

# 데이터 로드 및 전처리
datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]

def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

datasets = [clean_columns(df) for df in datasets]
all_data = pd.concat(datasets, ignore_index=True)

# -99 값을 NaN으로 변환
all_data[months] = all_data[months].replace(-99, np.nan)

# 파라미터 매핑 및 이름 수정
param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

unit_map = {
    'Precipitation': 'mm',
    'Number of Days with Precipitation ≥ 1 mm': '#Days',
    'Mean Daily Maximum Temperature': 'degC',
    'Mean Daily Minimum Temperature': 'degC',
    'Mean Daily Mean Temperature': 'degC',
    'Mean Sea Level Pressure': 'hPa',
    'Mean Vapor Pressure': 'hPa',
    'Total Number of Hours of Sunshine': 'Hours'
}

# 데이터에 Parameter 이름 적용
all_data['Parameter'] = all_data['Elem'].map(param_map)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 슬라이더 범위를 업데이트하는 함수
def update_sliders(selected_month, selected_param):
    # NaN을 무시한 최소/최대 값 계산
    filtered = all_data[all_data['Parameter'] == selected_param]
    min_value = filtered[selected_month].dropna().min()  # NaN 무시
    max_value = filtered[selected_month].dropna().max()
    param_name = selected_param.split(' (')[0]  # 이름에서 단위 제거
    unit = unit_map[param_name]
    return min_value, max_value, unit

# 데이터 필터링 및 지도 업데이트 함수
def update_plot(selected_month, selected_param, min_val, max_val):
    # 선택한 월과 파라미터로 데이터 필터링
    filtered = all_data[
        (all_data['Parameter'] == selected_param) &
        (all_data[selected_month] >= min_val) &
        (all_data[selected_month] <= max_val)
    ]

    # Plotly 지도 생성
    fig = px.scatter_geo(
        filtered,
        lat='Latitude',
        lon='Longitude',
        hover_name='Station',
        hover_data={'Country': True, selected_month: True, 'Parameter': True},
        projection='natural earth'
    )

    # 마커 및 스타일 업데이트
    fig.update_traces(
        mode='text',
        text='🚩',
        textfont=dict(size=12)
    )
    fig.update_geos(
        showland=True, landcolor="rgb(229, 229, 229)",
        showocean=True, oceancolor="rgb(204, 229, 255)",
        showcountries=True, showframe=False
    )
    fig.update_layout(
        title=f"{selected_month} - {selected_param} between {min_val} and {max_val}",
        margin={"r": 0, "t": 30, "l": 0, "b": 0}
    )

    fig.show()

# 위젯 구성
month_dropdown = widgets.Dropdown(
    options=months,
    value='Dec',
    description='Month:'
)

param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Mean Daily Maximum Temperature (degC)',
    description='Parameter:'
)

min_slider = widgets.IntSlider(value=15, min=0, max=50, step=1, description='Min Val:')
max_slider = widgets.IntSlider(value=20, min=0, max=50, step=1, description='Max Val:')

# 위젯 간 동적 연동
def on_param_or_month_change(change):
    # 선택한 Month와 Parameter 가져오기
    selected_month = month_dropdown.value
    selected_param = param_dropdown.value
    
    # 슬라이더의 min, max, 단위 업데이트
    min_value, max_value, unit = update_sliders(selected_month, selected_param)
    min_slider.min = int(min_value)
    min_slider.max = int(max_value)
    max_slider.min = int(min_value)
    max_slider.max = int(max_value)
    min_slider.value = int(min_value)
    max_slider.value = int(max_value)
    min_slider.description = f"Min Val ({unit})"
    max_slider.description = f"Max Val ({unit})"

# Month와 Parameter 변경 이벤트 리스너
month_dropdown.observe(on_param_or_month_change, names='value')
param_dropdown.observe(on_param_or_month_change, names='value')

# 최종 인터랙티브 구성
interact(
    update_plot,
    selected_month=month_dropdown,
    selected_param=param_dropdown,
    min_val=min_slider,
    max_val=max_slider
)